In [ ]:
#| label: libraries
#| include: false
import pandas as pd
import numpy as np
import plotly.express as px
import sqlite3

## Elevator pitch

_paste your elevator pitch here_
_A SHORT (4-5 SENTENCES) PARAGRAPH THAT `DESCRIBES KEY INSIGHTS` TAKEN FROM METRICS IN THE PROJECT RESULTS THINK TOP OR MOST IMPORTANT RESULTS._


In [ ]:
#| label: project data
#| code-summary: Read and format project data
# Include and execute your code here

sqlite_file = 'lahmansbaseballdb.sqlite'
con = sqlite3.connect(sqlite_file)

## QUESTION|TASK 1

__Write an SQL query to create a new dataframe about baseball players who attended BYU-Idaho. The new table should contain five columns: playerID, schoolID, salary, and the yearID/teamID associated with each salary. Order the table by salary (highest to lowest) and print out the table in your report.__


In [ ]:
#| label: Q1
#| code-summary: Read and format data
# Include and execute your code here

q = '''
    SELECT DISTINCT p.playerID
    ,      cp.schoolID
    ,      salary
    ,      s.yearID
    ,      s.teamID
    FROM people p
    LEFT JOIN collegeplaying cp
      ON cp.playerID = p.playerID
    LEFT JOIN salaries s
      ON p.playerID = s.playerID
    WHERE cp.schoolID = 'idbyuid'
    ORDER BY salary DESC
'''
# comment
results1 = pd.read_sql_query(q, con)

results1

## QUESTION|TASK 2

__This three-part question requires you to calculate batting average (number of hits divided by the number of at-bats)__

_a. Write an SQL query that provides playerID, yearID, and batting average for players with at least 1 at bat that year. Sort the table from highest batting average to lowest, and then by playerid alphabetically. Show the top 5 results in your report._


In [ ]:
#| label: Q2a
#| code-summary: Read and format data
# Include and execute your code here

q = '''
    SELECT playerID
    ,      yearID
    ,      ((1.0 * H) / AB) AS 'Batting_Average'
    FROM batting
    WHERE AB >= 1
    ORDER BY ((1.0 * H) / AB) DESC, playerid
    LIMIT 5
'''

results2 = pd.read_sql_query(q, con)

results2

_b. Use the same query as above, but only include players with at least 10 at bats that year. Print the top 5 results._


In [ ]:
#| label: Q2b
#| code-summary: Read and format data
# Include and execute your code here

q = '''
    SELECT playerID
    ,      yearID
    ,      ((1.0 * H) / AB) AS 'Batting_Average'
    FROM batting
    WHERE AB >= 10
    ORDER BY Batting_Average DESC, playerid
    LIMIT 5
'''

results3 = pd.read_sql_query(q, con)

results3

_c. Now calculate the batting average for players over their entire careers (all years combined). Only include players with at least 100 at bats, and print the top 5 results._


In [ ]:
#| label: Q2c
#| code-summary: Read and format data
# Include and execute your code here

q = '''
    SELECT playerID
    ,      SUM(H) AS 'Total_Hits'
    ,      SUM(AB) AS 'Total_At_Bats'
    ,      ((SUM(H) * 1.0) / SUM(AB)) AS 'Batting_Average'
    FROM batting
    GROUP BY playerID
    HAVING SUM(AB) >= 100
    LIMIT 5
'''

results4 = pd.read_sql_query(q, con)

results4


## QUESTION|TASK 3

__Pick any two baseball teams and compare them using a metric of your choice (average salary, home runs, number of wins, etc). Write an SQL query to get the data you need, then make a graph using Plotly Express to visualize the comparison. What do you learn?__


In [ ]:
#| label: Q3
#| code-summary: Read and format data
# Include and execute your code here

q1 = '''
    SELECT w, l
    FROM teams
    '''

results5 = pd.read_sql_query(q1, con)

results5

In [ ]:
#| label: Q3
#| code-summary: Read and format data
# Include and execute your code here

q = '''
    SELECT name
    ,      W AS 'Wins'
    ,      L AS 'Losses'
    ,      (SUM(W) + SUM(L)) AS 'Total_Games'
    ,      ((SUM(W) * 1.0) / (SUM(W) + SUM(L))) AS 'Win_Rate'
    FROM teams
    GROUP BY teamID
    ORDER BY Win_Rate DESC
'''

results5 = pd.read_sql_query(q, con)

results5

__Above is a chart of teams from the history of baseball. It consist of it's win/loss ratio to let us know how each team did against it's competition. Below is a scatter plot that shows us all of the teams and how they did with their Win Loss Ratio. As we can see, we have emphasized The Boston Red Sox and The New York Yankees. From what we can see in the graph, the New York Yankees tend to have more win per loss than the Boston Red Sox.__


In [ ]:
query = '''
    SELECT * 
    FROM teams
    '''

df['Team_Type'] = df['teamID'].apply(lambda x: 'Boston Red Sox' if x == 'BOS' else ('New York Yankees' if x == 'NYA' else 'Other'))

color_map = {'Boston Red Sox': 'red',
             'New York Yankees': 'blue',
             'Other': 'gray'}

fig = px.scatter(df, x='W', y='L', color='Team_Type', color_discrete_map=color_map, hover_name='yearID', hover_data={'W': True, 'L': True})

# Update layout for better visualization
fig.update_layout(title='Baseball Teams Win-Loss Scatter Plot',
                  xaxis_title='Wins', yaxis_title='Losses',
                  legend_title='Team', hovermode='closest')

fig.show()